航迹推算是根据先前估计的速度方向和行进时间对位置进行估计
但是更新所基于的测量值会收到系统和随机误差的影响

由于过程的递归性质，每个估计都是基于先前的测量，导致误差会随着时间推移而积累。在古代航海往往需要仅仅依靠指南针和计程仪（ship's log 测量速度）在海中航行数月，显然很难做到精确定位。

解决方案是一旦有机会就使用地理信息矫正位置估计。例如我们的眼前有两个灯塔，在不知道灯塔的距离的情况下，根据地图和指南针我们可以确定我们的位置应该在灯塔对应延长线的交点。

我们对定位问题的定义为真值x是确定但不可知的，我们对x的最佳估计为$\hat{x}$。同时我们还希望得到估计的不确定度用来衡量当前估计的好坏。

因此我们往往用所有可能位置上的PDF来表示机器人的位置估计。每个点的函数值对应车辆在该位置的可能性。车辆最可能在接近均值的位置，而峰值越凸出，标准差越小，车辆的位置估计越精准。使用PDF描述位置的另一个好处是我们可以同时表示多个可能位置。

推断问题也被称为逆向概率问题，通过处理依赖于x的数据，推断出x的值。对于我们的机器人定位问题，估计状态和真实状态的不确定度被表示为协方差矩阵。


估计机器人姿态的第一步是编写一个状态转移模型，它描述了车辆的状态如何从一个时间到下一个时间步的变化。
对于2D问题，我们的状态是朝向和位置，输入是当前时刻到下一时刻的移动距离和朝向变化。实际的机电输入是用户指定的车轮的转速和朝向，并通过传感器反馈的信息计算实际移动距离和朝向变化(里程计)。
<img src = "./discrete_model.png"/>
简化状态模型为
<img SRC = "./simple_discrete_model.png"/>

第二步是添加里程计噪声。由于编码器和地面打滑等问题，实际的位置和朝向和里程计计算并不完全相等。
<img SRC = "./odometry.png"/>
一般情况下将里程计噪声建模为均值为0的标准正态分布。


我们现在的问题是如何在给定前一个姿态和噪声里程计的情况下估计新的姿态。我们选择的工具是扩展kalman滤波。

对于本问题，状态为车辆的构型(x,y,$\theta$)


更新方程为：
<img SRC = "./prediction.png"/>

$x^{+}(k+1)$是根据0-k时刻状态和输入估计的k+1时刻的状态

$\hat{P}$是状态估计的协方差。$\hat{V}$是里程计噪声的协方差的估计值.

$F_x$和$F_v$为上面的状态更新函数对x和v的偏导数。
<img SRC = "./jacobian_F.png"/>


误差椭圆：

对于2D问题，只考虑机器人的位置，由于开始时假设x,y为标准正态分布，因此在状态更新过程中，状态的分布依然为高斯分布。
<img SRC = "./error_ellipses.png"/>
p(x)是一个在2D空间上分布的函数，在z轴处做一横截面，则横截面围成的图形为一个以$(\mu_x,\mu_y)$为中心的椭圆
显然，不确定度越小，椭圆围成的面积越小。椭圆的面积为det($P_{xy}$)，通过绘制其变化可以看到不确定性大小的变化情况。
<img SRC = "./det_P.png"/>
我们无法得知真实传感器的测量噪声协方差$V$，而是在更新过程中使用我们自己的估计噪声协方差$\hat{V}$，可以看到当我们的估计协方差大于实际协方差（蓝色），则不确定性增长速度要超过实际，反之，不确定性要小于实际。所以最好采取一些方案来确定协方差的适当值。



仅仅基于里程计更新的位置估计不确定度会不断增长，需要引入额外的信息来进行修正，例如地图的信息，车辆通过观察外界确认标志得到自身的位置。观测值和实际状态之间的关系为
<img src="./landmark.png">
获得观测值并不意味着我们知道了实际的状态，实际上观测值和里程计数据一样都是我们的一种测量，我们需要从中推断实际的位置。和之前相同这种观测同样带有观测误差。

已知在k+1时刻基于地图的观测为$z^{#}(k+1)$。而基于里程计估计得到的状态推导出的$z=h(\hat{x}^{+}(k+1),p_i)$。两者之间的差为：
<img src="./difference.png">
Kalman滤波使用这个更新的差来修正状态和不确定性：
<img src="./update.png">


## 四元数
四元数是复数的推广
$$
q = s<v_1,v_2,v_3>
$$
四元数满足加法，减法和数乘。这三种运算是逐个元素进行的。
四元数乘法为
$$
q_1 \circ q_2  = s_1 s_2 -v_1 v_2 <s_1 v_2 +s_2 v_1 + v_1 \times v_2>
$$
或者更简洁的矩阵形式

为了表示四元数对向量的旋转使用单位长度的四元数
$$
q = cos\frac{\theta}{2}<\hat{v}sin\frac{\theta}{2}>
$$
当使用q对向量进行旋转时，
$$
q\circ v\circ q^{-1}
$$
其中v为一个向量的四元数表示$0<v>$

当我们知道体坐标系下的角速度，希望知道dt后的姿态，标准做法是计算$e^[wdt]$，在dt较小时可以省略高阶项，得到增量旋转表示，相比完整的矩阵指数，避免了计算sin和cos，牺牲了少许精度。
$$
{^OR_B}(t+dt)={^OR_B} {e^{[^Bw]dt}} {^BR_O}[\hat{^Ox_B},\hat{^Oy_B},\hat{^Oz_B}] = {^OR_B} {e^{[^Bw]dt}} {^BR_O}{^OR_B(t)} = {^OR_B} {e^{[^Bw]dt}} =  {^OR_B}(t)*([^Bw]dt+I)
$$
但是从滤波器设计角度来看直接使用so3是不妥当的，因为so3的每一项并不是均匀变化的。一种思路是转换为欧拉角，但是欧拉角存在万向节锁问题，万向节锁的本质原因是在z转动为$\pi/2$的多解问题。这导致可能会出现角度跳变的问题。此外对于特别小的角度，欧拉角解算由于出现除以sin的操作，会出现数值奇异
$$
{^OR_B} = R_y(\theta_{pitch})R_z(\theta_{roll}=\pi/2)R_x(\theta_{yaw})=R_z(\pi/2)R_x(\theta_{pitch}+\theta_{yaw})
$$
使用四元数表示增量旋转得到的公式为
$$
q<k+1> = q<k>+dt/2*w\circ q<k>
$$
